In [1]:
from gnncloudmanufacturing.data import read_fatahi_dataset
from gnncloudmanufacturing.random_solution import random_solve
from gnncloudmanufacturing.validation import total_cost_from_graph, check_feasibility, total_cost_from_gamma
from gnncloudmanufacturing.utils import delta_from_gamma, graph_from_problem, gamma_from_target, delta_from_gamma
from gnncloudmanufacturing.graph_model import GNN, os_type, ss_type

import numpy as np
from tqdm.auto import trange, tqdm
from time import time
import pandas as pd
import torch

In [2]:
dataset = read_fatahi_dataset('../data/fatahi.xlsx')
for problem in dataset:
    print(f'Problem: {problem["name"]}')

  0%|          | 0/18 [00:00<?, ?it/s]

Problem: 5,10,10-1
Problem: 5,10,10-2
Problem: 5,10,10-3
Problem: 10,10,10-1
Problem: 10,10,10-2
Problem: 10,10,10-3
Problem: 5,10,20-1
Problem: 5,10,20-2
Problem: 5,10,20-3
Problem: 5,20,10-1
Problem: 5,20,10-2
Problem: 5,20,10-3
Problem: 5,20,20-1
Problem: 5,20,20-2
Problem: 5,20,20-3
Problem: 5,5,5-1
Problem: 5,5,5-2
Problem: 5,5,5-3


In [3]:
model = GNN.load_from_checkpoint(
    checkpoint_path="gnn_ver3.ckpt",
    ins_dim=1,
    ino_dim=1,
    out_dim=32,
    n_layers=2,
    lr=0.001,
    dropout_rate=0.2,
)
model.eval()

GNN(
  (convs): ModuleList(
    (0): AttnConvLayer(
      (W_s): Linear(in_features=1, out_features=32, bias=True)
      (W_os): Linear(in_features=3, out_features=32, bias=True)
      (W_ss): Linear(in_features=2, out_features=32, bias=True)
      (attn): Linear(in_features=64, out_features=1, bias=True)
      (W_in): Linear(in_features=1, out_features=32, bias=True)
      (W_self): Linear(in_features=1, out_features=32, bias=True)
      (W_out): Linear(in_features=1, out_features=32, bias=True)
      (W_o): Linear(in_features=96, out_features=32, bias=True)
    )
    (1): AttnConvLayer(
      (W_s): Linear(in_features=32, out_features=32, bias=True)
      (W_os): Linear(in_features=34, out_features=32, bias=True)
      (W_ss): Linear(in_features=33, out_features=32, bias=True)
      (attn): Linear(in_features=64, out_features=1, bias=True)
      (W_in): Linear(in_features=32, out_features=32, bias=True)
      (W_self): Linear(in_features=32, out_features=32, bias=True)
      (W_out):

In [4]:
problem_name = []
total_cost = []
comp_time = []
for problem in tqdm(dataset):
    start = time()
    total = np.inf
    for i in range(5):
        graph = graph_from_problem(problem)
        graph.edata['feat'][os_type][:, 0] /= 10
        graph.edata['feat'][ss_type][:] /= 100
        pred = model.predict(graph)
        gamma = gamma_from_target(pred, graph, problem)
        delta = delta_from_gamma(problem, gamma)
        check_feasibility(gamma, delta, problem)
        _total = total_cost_from_gamma(problem, gamma, delta).item()
        if total > _total:
            total = _total
    total_cost.append(total)
    problem_name.append(problem['name'])
    comp_time.append(time() - start)

  0%|          | 0/18 [00:00<?, ?it/s]

In [5]:
pd.DataFrame({'problem_name': problem_name, 'total_cost': total_cost, 'comp_time': comp_time}).round(2)

,problem_name,total_cost,comp_time
0,"5,10,10-1",10204.93,0.12
1,"5,10,10-2",11919.25,0.09
2,"5,10,10-3",12240.94,0.09
3,"10,10,10-1",21072.84,0.12
4,"10,10,10-2",24110.72,0.13
5,"10,10,10-3",27794.10,0.13
6,"5,10,20-1",10795.78,0.19
7,"5,10,20-2",11582.40,0.21
8,"5,10,20-3",16155.77,0.22
9,"5,20,10-1",27459.17,0.13
